In [1]:
from rembg import remove
from PIL import Image
import numpy as np

# Cargar imagen y eliminar fondo
with open('cat_dog_100/train/dog/dog.10095.jpg', 'rb') as f:
    input_image = f.read()

output_image = remove(input_image)

# Guardar el resultado sin fondo
with open('perro_sin_fondo.png', 'wb') as f:
    f.write(output_image)

ModuleNotFoundError: No module named 'rembg'

In [1]:
import numpy as np
import cv2
imagen = cv2.imread('cat_dog_100/train/dog/dog.10089.jpg', 0)
h, bins = np.histogram (imagen, bins = 256, range = [0, 256])
q = np.median(imagen)

print(imagen)
segmentada = imagen
segmentada[imagen>q] = 255
print(segmentada)

          
cv2.imshow('dst',segmentada)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[191 186 156 ... 124 149 166]
 [175 169 153 ... 135 146 154]
 [156 148 154 ... 134 130 129]
 ...
 [188 171 155 ... 187 169 132]
 [116 133 156 ... 192 186 157]
 [116 138 161 ... 212 223 215]]
[[255 255 156 ... 124 149 255]
 [255 255 153 ... 135 146 154]
 [156 148 154 ... 134 130 129]
 ...
 [255 255 155 ... 255 255 132]
 [116 133 156 ... 255 255 157]
 [116 138 255 ... 255 255 255]]


In [14]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 1. Preprocesamiento
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:  # Manejar imágenes no cargadas
        print(f"Error al cargar la imagen: {image_path}")
        return None
    image = cv2.resize(image, (128, 128))  # Redimensionar a 128x128
    image = cv2.equalizeHist(image)        # Ecualización del histograma
    return image

# 2. Extracción de características HOG
def extract_hog_features(image):

    fd = hog(
        image,
        orientations=8,
        pixels_per_cell=(16, 16),
        cells_per_block=(1, 1),
        visualize=False  # Solo devuelve el descriptor
    )
    return fd

# 3. Creación del dataset
def create_dataset(image_dir, feature_extractor):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []
    for label, class_name in enumerate(['cat', 'dog']):
        class_dir = os.path.join(image_dir, class_name)
        for file_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, file_name)
            image = preprocess_image(image_path)
            if image is None:  # Omitir imágenes no válidas
                continue
            features = feature_extractor(image)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)

def main_pipeline():
    # Definir rutas de datos
    train_dir = 'cat_dog_500/train'
    test_dir = 'cat_dog_500/test'

    # Crear datasets usando HOG
    print("Extrayendo características HOG...")
    X_train, y_train = create_dataset(train_dir, extract_hog_features)
    X_test, y_test = create_dataset(test_dir, extract_hog_features)

    # Normalización (opcional si las características no están en el mismo rango)
    X_train = (X_train - np.min(X_train)) / (np.max(X_train) - np.min(X_train))
    X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test))

    # Entrenamiento del modelo
    print("Entrenando modelo SVM...")
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)

    # Evaluación del modelo
    print("Evaluando modelo...")
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))

# Ejecutar el pipeline
if __name__ == "__main__":
    main_pipeline()


Extrayendo características HOG...
Entrenando modelo SVM...
Evaluando modelo...
              precision    recall  f1-score   support

           0       0.78      0.59      0.67       100
           1       0.67      0.83      0.74       100

    accuracy                           0.71       200
   macro avg       0.72      0.71      0.71       200
weighted avg       0.72      0.71      0.71       200



In [2]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 1. Preprocesamiento
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:  # Manejar imágenes no cargadas
        print(f"Error al cargar la imagen: {image_path}")
        return None
    image = cv2.resize(image, (128, 128))  # Redimensionar a 128x128
    image = cv2.equalizeHist(image)        # Ecualización del histograma
    return image

# 2. Extracción de características HOG
def extract_hog_features(image):

    fd = hog(
        image,
        orientations=8,
        pixels_per_cell=(16, 16),
        cells_per_block=(1, 1),
        visualize=False  # Solo devuelve el descriptor
    )
    return fd

# 3. Creación del dataset
def create_dataset(image_dir, feature_extractor):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []
    for label, class_name in enumerate(['cat', 'dog']):
        class_dir = os.path.join(image_dir, class_name)
        for file_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, file_name)
            image = preprocess_image(image_path)
            if image is None:  # Omitir imágenes no válidas
                continue
            features = feature_extractor(image)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)

from sklearn.metrics import precision_score, recall_score, f1_score

def clasificacion(y_true, y_pred):
    # Definir nombres de clases
    class_names = {0: "gatos", 1: "perros"}

    # Inicializar un diccionario para almacenar las métricas por clase
    report = {}
    for cls in [0, 1]:
        # Calcular las métricas por clase
        precision = precision_score(y_true, y_pred, labels=[cls], average="micro")
        report[class_names[cls]] = {
            'precision': precision
        }
    # Imprimir el reporte de clasificación manual
    print("Reporte de Clasificación :")
    for class_name, metrics in report.items():
        print(f"\nClase {class_name}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.2f}")

# Pipeline actualizado
def main_pipeline():
    train_dir = 'cat_dog_500/train'
    test_dir = 'cat_dog_500/test'

    print("Extrayendo características HOG...")
    X_train, y_train = create_dataset(train_dir, extract_hog_features)
    X_test, y_test = create_dataset(test_dir, extract_hog_features)

    print("Entrenando modelo SVM...")
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)

    print("Evaluando modelo...")
    y_pred = clf.predict(X_test)
    clasificacion(y_test, y_pred)


main_pipeline()

Extrayendo características HOG...
Entrenando modelo SVM...
Evaluando modelo...
Reporte de Clasificación :

Clase gatos:
  precision: 0.78

Clase perros:
  precision: 0.67


In [3]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# 1. Preprocesamiento
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:  # Manejar imágenes no cargadas
        print(f"Error al cargar la imagen: {image_path}")
        return None
    image = cv2.resize(image, (128, 128))  # Redimensionar a 128x128
    image = cv2.equalizeHist(image)        # Ecualización del histograma
    return image

# 2. Extracción de características HOG
def extract_hog_features(image):
    fd = hog(
        image,
        orientations=9,  # Aumentamos el número de orientaciones
        pixels_per_cell=(8, 8),  # Reducimos el tamaño de las celdas para capturar más detalles
        cells_per_block=(2, 2),  # Aumentamos el tamaño del bloque para mejorar la normalización
        visualize=False  # Solo devuelve el descriptor
    )
    return fd

# 3. Creación del dataset
def create_dataset(image_dir, feature_extractor):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []
    for label, class_name in enumerate(['cat', 'dog']):
        class_dir = os.path.join(image_dir, class_name)
        for file_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, file_name)
            image = preprocess_image(image_path)
            if image is None:  # Omitir imágenes no válidas
                continue
            features = feature_extractor(image)
            X.append(features)
            y.append(label)
    return np.array(X), np.array(y)

# 4. Clasificación
def clasificacion(y_true, y_pred):
    print(classification_report(y_true, y_pred, target_names=["gatos", "perros"]))

# Pipeline actualizado con búsqueda de hiperparámetros (GridSearchCV)
def main_pipeline():
    train_dir = 'cat_dog_500/train'
    test_dir = 'cat_dog_500/test'

    print("Extrayendo características HOG...")
    X_train, y_train = create_dataset(train_dir, extract_hog_features)
    X_test, y_test = create_dataset(test_dir, extract_hog_features)

    print("Normalizando características...")
    # Normalizamos las características
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print("Realizando búsqueda de hiperparámetros con GridSearchCV...")
    parameters = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
    clf = SVC(kernel='rbf')
    grid_search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Obtener los mejores parámetros
    print("Mejores parámetros encontrados:", grid_search.best_params_)

    # Evaluando el modelo con los mejores parámetros
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    clasificacion(y_test, y_pred)

# Ejecutar el pipeline mejorado
main_pipeline()


Extrayendo características HOG...
Normalizando características...
Realizando búsqueda de hiperparámetros con GridSearchCV...
Mejores parámetros encontrados: {'C': 1, 'gamma': 'scale'}
              precision    recall  f1-score   support

       gatos       0.99      0.77      0.87       100
      perros       0.81      0.99      0.89       100

    accuracy                           0.88       200
   macro avg       0.90      0.88      0.88       200
weighted avg       0.90      0.88      0.88       200



In [5]:
import cv2
import numpy as np
import os
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score

# 1. Preprocesamiento
def preprocesar_imagen(ruta_imagen):
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    if imagen is None:  # Manejar imágenes no cargadas
        print(f"Error al cargar la imagen: {ruta_imagen}")
        return None
    imagen = cv2.resize(imagen, (128, 128))  # Redimensionar a 128x128
    imagen = cv2.equalizeHist(imagen)        # Ecualización del histograma
    return imagen

# 2. Extracción de características HOG
def extraer_caracteristicas_hog(imagen):
    fd = hog(
        imagen,
        orientations=9,  # Aumentamos el número de orientaciones
        pixels_per_cell=(8, 8),  # Reducimos el tamaño de las celdas para capturar más detalles
        cells_per_block=(2, 2),  # Aumentamos el tamaño del bloque para mejorar la normalización
        visualize=False  # Solo devuelve el descriptor
    )
    return fd

# 3. Creación del dataset
def crear_dataset(ruta_imagenes, extractor_caracteristicas):
    """
    Crea un dataset de características y etiquetas a partir de un directorio.
    """
    X, y = [], []
    for etiqueta, nombre_clase in enumerate(['cat', 'dog']):
        directorio_clase = os.path.join(ruta_imagenes, nombre_clase)
        for nombre_archivo in os.listdir(directorio_clase):
            ruta_imagen = os.path.join(directorio_clase, nombre_archivo)
            imagen = preprocesar_imagen(ruta_imagen)
            if imagen is None:  # Omitir imágenes no válidas
                continue
            caracteristicas = extractor_caracteristicas(imagen)
            X.append(caracteristicas)
            y.append(etiqueta)
    return np.array(X), np.array(y)

# 4. Clasificación
def clasificacion(y_real, y_predicha):
    nombres_clases = {0: "gatos", 1: "perros"}

    # Inicializar un diccionario para almacenar las métricas por clase
    reporte = {}
    for clase in [0, 1]:
        # Calcular las métricas por clase
        precision = precision_score(y_real, y_predicha, labels=[clase], average="micro")
        reporte[nombres_clases[clase]] = {
            'precisión': precision
        }
    # Imprimir clasificación
    print("Clasificación:")
    for nombre_clase, metricas in reporte.items():
        print(f"\nClase {nombre_clase}:")
        for metrica, valor in metricas.items():
            print(f"  {metrica}: {valor:.2f}")

# Pipeline actualizado con búsqueda de hiperparámetros (GridSearchCV)
def pipeline_principal():
    ruta_train = 'cat_dog_500/train'
    ruta_test = 'cat_dog_500/test'

    print("Extrayendo características HOG...")
    X_train, y_train = crear_dataset(ruta_train, extraer_caracteristicas_hog)
    X_test, y_test = crear_dataset(ruta_test, extraer_caracteristicas_hog)

    print("Normalizando características...")
    # Normalizamos las características
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    print("Realizando búsqueda de hiperparámetros con GridSearchCV...")
    parametros = {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}
    clf = SVC(kernel='rbf')
    busqueda_grid = GridSearchCV(clf, parametros, cv=5, scoring='accuracy')
    busqueda_grid.fit(X_train, y_train)

    # Evaluando el modelo con los mejores parámetros
    mejor_modelo = busqueda_grid.best_estimator_
    y_predicha = mejor_modelo.predict(X_test)
    clasificacion(y_test, y_predicha)

# Ejecutar el pipeline mejorado
pipeline_principal()


Extrayendo características HOG...
Normalizando características...
Realizando búsqueda de hiperparámetros con GridSearchCV...
Mejores parámetros encontrados: {'C': 1, 'gamma': 'scale'}
Clasificación:

Clase gatos:
  precisión: 0.99

Clase perros:
  precisión: 0.81


In [9]:
#Pruebas harrys
import cv2
import numpy as np
imagen = cv2.imread("cat_dog_100/test/cat/cat.10003.jpg", 0)
print(imagen.shape)
imagen = cv2.cornerHarris(imagen,2,3,0.04) 
imagen = np.uint8(imagen)
print(np.max(imagen))
print(imagen.shape)
print(imagen)

(375, 499)
0
(375, 499)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [22]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def harris_corner_detection(image_path):
    # Cargar la imagen en escala de grises
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Convertir a tipo float32
    gray = np.float32(gray)

    # Aplicar el detector de esquinas de Harris
    dst = cv2.cornerHarris(src=gray, blockSize=2, ksize=3, k=0.04)
    posiciones, valores = maximos_locales(dst)
    print(valores.shape)
    posiciones = posiciones[valores > 0.01 * np.max(dst)]
    valores = valores[valores > 0.01 * np.max(dst)]
    print(valores.shape)
    print(posiciones.shape)
    reducir_mat2(8, dst)
    print(len(dst))
# Ruta de la imagen (reemplázala con la ruta a tu imagen)
image_path = "cat_dog_100/test/cat/cat.10003.jpg"
harris_corner_detection(image_path)


(261,)
(46,)
(46, 2)
[[ 4.61059928e-01  7.91693181e-02  4.90594053e+00  4.65978719e+05
  -3.93766031e+05  1.13417800e+06  1.20932977e+05  2.35521031e+05]
 [ 1.32167000e+05  5.58460156e+03 -3.75312477e-01  2.30647900e+06
  -2.02303812e+05  2.94167500e+05  1.02522575e+06  1.20855000e+06]
 [ 1.67733960e+03  5.32492041e+03 -5.25376289e+04  1.87280625e+06
   3.03566650e+06  7.88391438e+05  7.84398375e+05  9.59538062e+05]
 [ 9.64690234e+03  1.02857598e+04 -1.15484650e+06  8.52559000e+05
   1.09412040e+07  1.42764625e+05  2.53263062e+05  1.56362469e+05]
 [ 1.30747807e+00  2.63560229e+03 -4.13080625e+05 -1.77883225e+06
   7.76082812e+04  4.27502109e+04 -1.57965391e+05 -5.07622852e+04]
 [ 8.08658695e+00 -1.12836599e+00 -1.08131288e+06 -5.66537438e+05
  -1.80576147e+03  1.69197969e+04 -1.90720012e+06 -4.36385219e+05]
 [ 1.67506289e+04 -1.39084863e+04  1.09501270e+04 -5.76452562e+05
   1.89528477e+04  1.08801230e+04 -1.51936547e+05 -4.58438633e+04]
 [-4.30090312e+04 -1.04779414e+05 -1.32314531e+0

In [6]:
from scipy.ndimage import maximum_filter
def maximos_locales(matriz):
    # Aplicar un filtro de máxima vecindad 3x3
    vecinos_maximos = maximum_filter(matriz, size=3, mode='constant', cval=0)
    
    # Identificar dónde los valores coinciden con el filtro (máximos locales)
    maximos = (matriz == vecinos_maximos)
    
    # Ignorar valores cero si es necesario
    maximos = maximos & (matriz > 0)
    
    # Obtener las posiciones de los máximos locales
    posiciones = np.argwhere(maximos)
    
    return posiciones, matriz[maximos]

In [5]:
def reducir_mat(tamaño, posiciones, valores):
    filas = tamaño
    columnas = tamaño
    print(posiciones[0])
    mat = np.array([[tamaño], [tamaño]])
    fila_ant =  0
    col_ant = 0
    for i in range(len(posiciones)):
        vect = np.array([])
        fila = posiciones[i][0] % tamaño
        columna = posiciones[i][1] % tamaño
        if (fila == fila_ant and columna == col_ant):
            vect.append(valores[fila, columna])
        else:
            if(np.isnan(vect)):
                vect = 0
            mat[fila_ant][col_ant] = np.median(vect)
            vect.append(valores[fila, columna])
            
        # Asignamos el valor a la matriz en las posiciones calculadas
        fila_ant = fila
        col_ant = columna
    print(mat)


In [6]:
import numpy as np

def reducir_mat(tamaño, posiciones, valores):
    # Crear una matriz de ceros (sin usar acumuladores de listas)
    mat = np.zeros((tamaño, tamaño), dtype=float)

    # Crear un diccionario para agrupar los valores por posiciones únicas
    acumulador = {}

    # Agrupar los valores según fila % tamaño y columna % tamaño
    for i in range(len(posiciones)):
        fila = posiciones[i][0] % tamaño
        columna = posiciones[i][1] % tamaño
        if (fila, columna) not in acumulador:
            acumulador[(fila, columna)] = []
        acumulador[(fila, columna)].append(valores[i])

    # Calcular la mediana para cada celda con valores
    for (fila, columna), vals in acumulador.items():
        mat[fila, columna] = np.median(vals)

    # Mostrar la matriz resultante
    print("Matriz resultante:")
    print(mat)
    return mat


In [42]:
import cv2
import numpy as np
image_path = "cat_dog_100/test/cat/cat.10003.jpg"
imagen = cv2.imread(image_path, 0)
imagen = np.resize(imagen, (128, 128))
sift = cv2.SIFT_create(contrastThreshold=0.04, edgeThreshold=15)
keypoints, descriptors = sift.detectAndCompute(imagen, None)
print(keypoints[0].pt)
print(keypoints[0].response)
print(np.max(keypoints.response))
print(descriptors)


(10.079602241516113, 76.81034851074219)
0.02941621094942093


AttributeError: 'tuple' object has no attribute 'response'

In [21]:
import numpy as np

def reducir_mat2(tamaño, matriz):
    # Crear una matriz de ceros (sin usar acumuladores de listas)
    mat = np.zeros((tamaño, tamaño), dtype=float)
    filas_sub, columnas_sub = tamaño, tamaño

    # Crear un diccionario para agrupar los valores por posiciones únicas
    acumulador = {}

    # Agrupar los valores según fila % tamaño y columna % tamaño
    submatrices = [matriz[i:i+filas_sub, j:j+columnas_sub]
               for i in range(0, matriz.shape[0], filas_sub)
               for j in range(0, matriz.shape[1], columnas_sub)]

    # Calcular la mediana para cada celda con valores
    medias = [np.mean(submatriz) for submatriz in submatrices]
    
    # Crear una nueva matriz de medias (2x2)
    matriz_medias = np.array(medias).reshape(tamaño, tamaño)
    print(matriz_medias)
    return mat


In [15]:
import numpy as np

# Matriz original
matriz = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 0, 1, 2],
    [3, 4, 5, 6]
])

# Tamaño de las submatrices
filas_sub, columnas_sub = 2, 2

# Dividir la matriz en submatrices
submatrices = [matriz[i:i+filas_sub, j:j+columnas_sub]
               for i in range(0, matriz.shape[0], filas_sub)
               for j in range(0, matriz.shape[1], columnas_sub)]

# Imprimir las submatrices
for idx, submatriz in enumerate(submatrices, start=1):
    print(f"Submatriz {idx}:\n{submatriz}\n")


Submatriz 1:
[[1 2]
 [5 6]]

Submatriz 2:
[[3 4]
 [7 8]]

Submatriz 3:
[[9 0]
 [3 4]]

Submatriz 4:
[[1 2]
 [5 6]]

